In [1]:
import os
import pandas as pd
import imageio

In [2]:
def head_sensor(video_path, csv_path, output_folder, f_frame_count=32, l_frame_count=16):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]
    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']

    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0

    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'BLUE_HEAD_HIT') | (df['matchLogItemType'] == 'RED_HEAD_HIT')].iterrows():
        
        system_time = row['systemTime']
      

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        video_time_seconds = (system_time - start_system_time) / 1000.0
       

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        frame_number = int(video_time_seconds * fps)
      
        frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme
     

        # İlk frame'den önceki ve sonraki frame'leri al
        start_frame = max(0, frame_number - f_frame_count)
  
        end_frame = min(frame_count - 1, frame_number + l_frame_count)
        c = c+1

        # Videoyu oku ve yeni videoya yaz
        output_video = []
        
        try:
            for i in range(start_frame, end_frame + 1):
                frame = video.get_data(i)
                output_video.append(frame)
             
        except IndexError as e:
            c = c-1
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")

        # Klasör kontrolü ve oluşturma
        os.makedirs(output_folder, exist_ok=True)
        
        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'head_sensor_{cam_num}_cam_{index+2}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    print(f'Sensorden algilanan kafa vuruslari {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')
    

In [3]:
import random
def random_video(video_path, csv_path, output_folder, f_frame_count=32, l_frame_count=16):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]
    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']

    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0

    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'BLUE_BODY_POINT') | (df['matchLogItemType'] == 'RED_BODY_POINT')].iterrows():
        
        system_time = row['systemTime']
      

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        video_time_seconds = (system_time - start_system_time) / 1000.0
       

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        frame_number = int(video_time_seconds * fps)
      
        frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme
     

        # İlk frame'den önceki ve sonraki frame'leri al
        start_frame = max(0, frame_number - f_frame_count)
  
        end_frame = min(frame_count - 1, frame_number + l_frame_count)
        c = c+1
        
        rastgele_sayi = random.randint(60, 150)
        print("Üretilen rastgele sayı:", rastgele_sayi)

        # Videoyu oku ve yeni videoya yaz
        output_video = []
        
        try:
            for i in range(start_frame+rastgele_sayi, end_frame + 1+rastgele_sayi):
                frame = video.get_data(i)
                output_video.append(frame)
             
        except IndexError as e:
            c = c-1
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")

        # Klasör kontrolü ve oluşturma
        os.makedirs(output_folder, exist_ok=True)
        
        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'random_video_{cam_num}_cam_{index+2}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    print(f'Rasgele vuruslar {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')
    

In [4]:
def head_scoreboard(video_path, csv_path, output_folder, f_frame=64):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0

    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df.iterrows():
        match_log_type = row['matchLogItemType']
        entry_value = row['entryValue']

        if (match_log_type == 'BLUE_HEAD_POINT' or match_log_type == 'RED_HEAD_POINT') and entry_value == 'SCOREBOARD_EDITOR':
            system_time = row['systemTime']

            # TIMEOUT satırını bulma
            timeout_row = None
            for i in range(index - 1, -1, -1):
                if df.at[i, 'matchLogItemType'] == 'TIMEOUT':
                    timeout_row = df.iloc[i]
                    break

            if timeout_row is not None:
                timeout_system_time = timeout_row['systemTime']

                # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
                video_time_seconds = (timeout_system_time - start_system_time) / 1000.0

                # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
                frame_number = int(video_time_seconds * fps)
                frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

                # 5 saniye önceki frame'leri al
                start_frame = max(0, frame_number - int(f_frame))
                end_frame = min(frame_count - 1, frame_number)
                c = c+1
                # Videoyu oku ve yeni videoya yaz
                output_video = []
                try:
                    for i in range(start_frame, end_frame + 1):
                        frame = video.get_data(i)
                        output_video.append(frame)
                except IndexError as e:
                    print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
                    c = c-1
                # Klasör kontrolü ve oluşturma
                os.makedirs(output_folder, exist_ok=True)

                # Oluşturulan videoyu belirtilen klasöre kaydet
                output_path = os.path.join(output_folder, f'head_scoreboard_{cam_num}_cam_{index}.mkv')
                output_writer = imageio.get_writer(output_path, fps=fps)
                for frame in output_video:
                    output_writer.append_data(frame)
                output_writer.close()
    print(f'Scoreboardan eklenen kafa vuruslari {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')
    print(fps)


In [5]:
def rotation_judge(video_path, csv_path, output_folder, f_frame_count=64, l_frame_count=0):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0
    
    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'RED_JUDGE_BODY_TECH') | (df['matchLogItemType'] == 'BLUE_JUDGE_BODY_TECH') | (df['matchLogItemType'] == 'RED_JUDGE_BODY_TECH') | (df['matchLogItemType'] == 'RED_JUDGE_HEAD_TECH')].iterrows():
        system_time = row['systemTime']

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        video_time_seconds = (system_time - start_system_time) / 1000.0

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        frame_number = int(video_time_seconds * fps)
        frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

        # Belirtilen sayıda önceki frame'leri al
        
        start_frame = max(0, frame_number - f_frame_count)
        end_frame = min(frame_count - 1, frame_number + l_frame_count)
        c = c+1
        # Videoyu oku ve yeni videoya yaz
        output_video = []
        try:
            for i in range(start_frame, end_frame + 1):
                frame = video.get_data(i)
                output_video.append(frame)
        except IndexError as e:
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
            c = c-1

        # Klasör kontrolü ve oluşturma
        os.makedirs(output_folder, exist_ok=True)

        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'rotation_judge_{cam_num}_cam_{index}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    print(f'Yan hakemlerin isaretledikleri rotasyonlu vuruslar {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')

In [6]:

def rotation_scoreboad(video_path, csv_path, output_folder, f_sec=5):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0
    
    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df.iterrows():
        match_log_type = row['matchLogItemType']
        entry_value = row['entryValue']

        if (match_log_type in ['RED_BODY_TECH_POINT', 'RED_HEAD_TECH_POINT', 'BLUE_BODY_TECH_POINT', 'BLUE_HEAD_TECH_POINT']) and entry_value == 'SCOREBOARD_EDITOR':
            system_time = row['systemTime']

            # TIMEOUT satırını bulma
            timeout_row = None
            for i in range(index - 1, -1, -1):
                if df.at[i, 'matchLogItemType'] == 'TIMEOUT':
                    timeout_row = df.iloc[i]
                    break

            if timeout_row is not None:
                timeout_system_time = timeout_row['systemTime']

                # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
                video_time_seconds = (timeout_system_time - start_system_time) / 1000.0

                # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
                frame_number = int(video_time_seconds * fps)
                frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

                # Belirtilen sayıda önceki frame'leri al
                start_frame = max(0, frame_number - int(f_sec * fps))
                end_frame = min(frame_count - 1, frame_number)
                c = c+1
                # Videoyu oku ve yeni videoya yaz
                output_video = []
                try:
                    for i in range(start_frame, end_frame + 1):
                        frame = video.get_data(i)
                        output_video.append(frame)
                except IndexError as e:
                    print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
                    c = c-1

                # Klasör kontrolü ve oluşturma
                os.makedirs(output_folder, exist_ok=True)

                # Oluşturulan videoyu belirtilen klasöre kaydet
                output_path = os.path.join(output_folder, f'rotation_scoreboad_{cam_num}_cam_{index}.mkv')
                output_writer = imageio.get_writer(output_path, fps=fps)
                for frame in output_video:
                    output_writer.append_data(frame)
                output_writer.close()
    print(f'Scoreboardan verilen rotasyonlu vuruslar {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')

In [7]:



def punch_judge(video_path, csv_path, output_folder, f_frame_count=92, l_frame_count=0):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0
    
    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'RED_JUDGE_PUNCH') | (df['matchLogItemType'] == 'BLUE_JUDGE_PUNCH')].iterrows():
        system_time = row['systemTime']

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        video_time_seconds = (system_time - start_system_time) / 1000.0

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        frame_number = int(video_time_seconds * fps)
        frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

        # Belirtilen sayıda önceki frame'leri al
        
        start_frame = max(0, frame_number - f_frame_count)
        end_frame = min(frame_count - 1, frame_number + l_frame_count)
        c = c+1
        # Videoyu oku ve yeni videoya yaz
        output_video = []
        try:
            for i in range(start_frame, end_frame + 1):
                frame = video.get_data(i)
                output_video.append(frame)
        except IndexError as e:
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
            c = c-1

        # Klasör kontrolü ve oluşturma
        os.makedirs(output_folder, exist_ok=True)

        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'punch_judge_{cam_num}_cam_{index}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    print(f'Yan hakemlerin isaretledikleri yumruk vurusları {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')

In [8]:



def punch_judge(video_path, csv_path, output_folder, f_frame_count=92, l_frame_count=0):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0
    
    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'BLUE_ADD_GAME_JEON') | (df['matchLogItemType'] == 'BLUE_ADD_GAME_JEON')].iterrows():
        system_time = row['systemTime']

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        video_time_seconds = (system_time - start_system_time) / 1000.0

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        frame_number = int(video_time_seconds * fps)
        frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

        # Belirtilen sayıda önceki frame'leri al
        
        start_frame = max(0, frame_number - f_frame_count)
        end_frame = min(frame_count - 1, frame_number + l_frame_count)
        c = c+1
        # Videoyu oku ve yeni videoya yaz
        output_video = []
        try:
            for i in range(start_frame, end_frame + 1):
                frame = video.get_data(i)
                output_video.append(frame)
        except IndexError as e:
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
            c = c-1

        # Klasör kontrolü ve oluşturma
        os.makedirs(output_folder, exist_ok=True)

        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'game_jeon_{cam_num}_cam_{index}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    print(f'Yan hakemlerin verdikleri Game-Jeonlar {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')

In [18]:
date='02_13'
court_num = 3
cam_num = 'live'

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/AntalyaKodlari/downloads'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = '11th TURKISH OPEN KYORUGI - G- 1 13 FEBRUARY 2024 - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/matchlog'
csv_name = '02_13-320-334.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Ham Video/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/cam_{cam_num}/{court_num}th_court_{cam_num}_cam_{date}'

In [16]:
date='02_13'
court_num = 3
cam_num = 'live'

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/AntalyaKodlari/downloads'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = '11th TURKISH OPEN KYORUGI - G- 1 13 FEBRUARY 2024 - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/3th_court/matchlog'
csv_name = '02_13-301-319.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Ham Video/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/cam_{cam_num}/{court_num}th_court_{cam_num}_cam_{date}'

In [69]:
date='02_14'
court_num = 3
cam_num = 'live'

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/AntalyaKodlari/downloads'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = '11th TURKISH OPEN KYORUGI - G- 1 14 FEBRUARY 2024 - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/3th_court/matchlog'
csv_name = '02_14-307-308_zaman.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Ham Video/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/cam_{cam_num}/{csv_name}_{court_num}th_court_{cam_num}_cam'

In [26]:
date='08_02'
court_num = 3
cam_num = 1

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-1'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-2'
csv_name = '08.02-347-1345.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Ham Video/Turkish_Open_2024_Antalya/Konya/{date}_2024/{court_num}th_court/cam_{cam_num}/{court_num}th_court_{cam_num}_cam_{date}'

In [25]:
date = '05_26'
court_num = 3

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/Videolar/multigames'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/3th_court'
csv_name = '05_26-301-316'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [32]:
date = '05_26'
court_num = 3

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/Videolar/multigames'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/3th_court'
csv_name = '05_26-318-322'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [40]:
date = '05_26'
court_num = 3

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/Videolar/multigames'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/3th_court'
csv_name = '05_26-323-331'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [49]:
date = '05_26'
court_num = 4

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/Videolar/multigames'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 4 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/4th_court'
csv_name = '05_26-401-421'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [53]:
date = '05_26'
court_num = 4

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/Videolar/multigames'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 4 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/4th_court'
csv_name = '05_26-422-432'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [61]:
date = '05_26'
court_num = 5

video_konum  ='/home/muhlabws4/Videolar/multigames'

video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 5 COURT.mp4'
video_path =f'{video_konum}/{video_name}'


csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/5th_court'
csv_name = '05_26-501-519'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [64]:
date = '05_26'
court_num = 5

video_konum  ='/home/muhlabws4/Videolar/multigames'

video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 26052024 - 3st DAY - 5 COURT.mp4'
video_path =f'{video_konum}/{video_name}'


csv_konum = '/home/muhlabws4/Belgeler/GitHub/taeakwondoProject/Log File/Multigames/birlestirilmis/05_26_2024/5th_court'
csv_name = '05_26-520-526'
csv_path = f'{csv_konum}/{csv_name}.csv'

output_path = f'/home/muhlabws4/Ham Video/Multigames/{date}_2024/{court_num}th_court/{court_num}th_court_{csv_name}'

In [65]:
rotation_judge(video_path, csv_path, f'{output_path}_rotation_judge', f_frame_count=30)

Yan hakemlerin isaretledikleri rotasyonlu vuruslar /home/muhlabws4/Ham Video/Multigames/05_26_2024/5th_court/5th_court_05_26-520-526_rotation_judge dosyasina kaydedildi. Toplam 5 pozisyon.


In [41]:
random_video(video_path, csv_path, f'{output_path}_random_video', f_frame_count=32,l_frame_count=31,)

Üretilen rastgele sayı: 108
Üretilen rastgele sayı: 121
Üretilen rastgele sayı: 82
Üretilen rastgele sayı: 80
Üretilen rastgele sayı: 63
Üretilen rastgele sayı: 82
Üretilen rastgele sayı: 95
Üretilen rastgele sayı: 92
Üretilen rastgele sayı: 141
Üretilen rastgele sayı: 105
Üretilen rastgele sayı: 119
Üretilen rastgele sayı: 142
Üretilen rastgele sayı: 82
Üretilen rastgele sayı: 107
Üretilen rastgele sayı: 74
Üretilen rastgele sayı: 104
Üretilen rastgele sayı: 63
Üretilen rastgele sayı: 69
Üretilen rastgele sayı: 66
Rasgele vuruslar /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video dosyasina kaydedildi. Toplam 19 pozisyon.


In [42]:
punch_judge(video_path, csv_path, f'{output_path}_punch_judge', f_frame_count=62)

Yan hakemlerin isaretledikleri yumruk vurusları /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_punch_judge dosyasina kaydedildi. Toplam 16 pozisyon.


In [43]:
head_sensor(video_path, csv_path, f'{output_path}_head_sensor', f_frame_count=32,l_frame_count=31,)

Hata: 98630. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.
Hata: 98997. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.
Hata: 100388. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.
Sensorden algilanan kafa vuruslari /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_head_sensor dosyasina kaydedildi. Toplam 9 pozisyon.


In [44]:
head_scoreboard(video_path, csv_path, f'{output_path}_head_scoreboard', f_frame=128) 

Hata: 95580. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.
Scoreboardan eklenen kafa vuruslari /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_head_scoreboard dosyasina kaydedildi. Toplam 3 pozisyon.
30.3


In [46]:
rotation_scoreboad(video_path, csv_path, f'{output_path}_rotation_scoreboard', f_sec=7)

Scoreboardan verilen rotasyonlu vuruslar /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_rotation_scoreboard dosyasina kaydedildi. Toplam 1 pozisyon.


In [22]:
import cv2
cap = cv2.VideoCapture("/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_punch_judge/punch_judge_live_cam_132_320-334.mkv")
fps = cap.get(cv2.CAP_PROP_FPS)
toplam_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Toplam Frame Sayısı: {toplam_frame}")
print(f"FPS Değeri: {fps}")

Toplam Frame Sayısı: 108
FPS Değeri: 30.0
